<img src="../../Img/backdrop-wh.png" alt="Drawing" style="width: 300px;"/>  

# Word Embeddings

* * * 

<div class="alert alert-success">  
    
### Learning Objectives 
    
* Understand word embeddings, and how they are used in information retrieval.
* Use `gensim`'s `word2vec` method to create word vectors for a corpus.
* Use word embeddings to calculate word similarity.
* Use word embeddings to reflect on implicit biases in your data.
</div>

### Icons Used in This Notebook
🔔 **Question**: A quick question to help you understand what's going on.<br>
💡 **Tip**: How to do something a bit more efficiently or effectively.<br>
⚠️ **Warning:** Heads-up about tricky stuff or common mistakes.<br>
💭 **Reflection**: Reflecting on ethical implications, biases, and social impact in data science.<br>

### Sections
1. [Word Embeddings](#we)
2. [Using Comments Data](#comments)
3. [Constructing a Word2Vec Model](#w2v)
4. [Word Similarity](#sim)
5. [Exploring Language Biases With Word Embeddings](#bias)


<a id='we'></a>

# Word Embeddings

The goal of word embedding models is to learn **numerical representations** of text corpora. Like with TF-IDF, it's all about how we construct that numerical representation.

When using word embeddings, we look for a **vector** of numbers to represent each term. The actual numbers themselves won't be meaningful to us as humans. However, if successful, the vectors for each term should encode information about the meaning or concept the term represents, as well as the relationship between it and other terms in the vocabulary.

In this notebook, we'll work with word embeddings using Word2Vec.

## Word2Vec

**Word2Vec** is one example of a word embeddings model. It learns by taking words and their contexts (e.g. sentences) into account, and can then try to predict other words. Given enough data, usage and contexts, Word2Vec can make accurate guesses about a word’s meaning based on its appearances. Those guesses can be used to establish a word’s association with other words (e.g. "Paris" is to "France" as “Berlin” is to “Germany”), or cluster documents and classify them by topic.

<img src="../../img/we.png" alt="Word Embeddings" width="600"/>

Word vector models such as Word2Vec are fully **unsupervised**: they learn all of these meanings and relationships without any advance knowledge. Unsupervised learning requires the specification of a right task. We won't go into detail in this lesson, but you can roughly think of the task as predicting nearby words, given a specific word. Read [this post](https://tomvannuenen.medium.com/analyzing-reddit-communities-with-python-part-6-word-embeddings-f92bba876d60) if you want a deeper introduction.

We will be using the `gensim` package, which offers Word2Vec.

In [1]:
# Package imports
import os
import pandas as pd
import numpy as np

import pickle
from gensim.models import Word2Vec
import multiprocessing

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

<a id='comments'></a>

# Using Comments Data

As we will be considering the language biases in the next notebook, we will use the **comments** of the r/amitheasshole subreddit this time. The thinking behind this is that this data will be derived from more people, and include more evaluative statements. After all, comments on r/amitheasshole generally evaluate the original posts.

💭 **Reflection**: Whether you want to be using submissions or comments for your own data depends on the question you are asking of that data! Make sure to think about this and discuss it with teammates.

To save us some time, we have done the preprocessing for you and saved the trigrams (a list of lists) in a pickle file.

In [2]:
with open('../../data/aita_comment_trigrams.pickle', 'rb') as f:
    # Load the object from the file
    comments = pickle.load(f)

In [3]:
# Split into lists of words for Word2Vec
comment_list = [comment.split() for comment in comments]
comment_list[0]

['like',
 'genuine',
 'reason',
 'upset',
 'question',
 'sound',
 'people',
 'ask',
 'curious',
 'direct',
 'question',
 'superior']

**Use the `3_Preprocessing_Project.ipynb` notebook if you want to preprocess your own comment data.**

**Use the `6_Word_Embeddings_Project.ipynb` notebook to run the Word2Vec operations explained in this notebook on your own data.**

<a id='w2v'></a>

# Constructing a Word2Vec Model

Let's create our word embeddings model. The input to this model is our corpus split up into words. The model's output is a set of "vectors" (one for each word) in $N$ dimensions (we choose $N$ before creating the model). Think of these vectors as "features", capturing latent meaning.

This model allows us to group the vectors of similar words together in vector space. We can then reduce the dimensionality to visualize the results in a way humans can understand (such as in a 2-dimensional space), or to perform linear algebra operations in order to find out to what extent words are related.

We now instantiate and train our Word2Vec model, using the parameters below.

In [4]:
cores = multiprocessing.cpu_count() # Number of cores at your disposal

n_features = 300     # Word vector dimensionality (how many features each word will be given)
min_word_count = 10  # Minimum word count to be taken into account
n_workers = cores    # Number of threads to run in parallel (equal to your amount of cores)
window = 5           # Context window size
downsampling = 1e-2  # Downsample setting for frequent words
seed = 1             # Seed for the random number generator (to create reproducible results)
sg = 1               # Skip-gram = 1, CBOW = 0
epochs = 20          # Number of iterations over the corpus

model = Word2Vec(
    sentences=comment_list,
    workers=n_workers,
    vector_size=n_features,
    min_count=min_word_count,
    window=window,
    sample=downsampling,
    seed=seed,
    sg=sg)

That was it! We have a Word Embeddings model now. Let's save it so that we don't have to train it again. Then, we can reload the embeddings so that we don't have to train it every single time:

In [5]:
model.save('../../data/aita_comments.emb')

In [6]:
model = Word2Vec.load('../../data/aita_comments.emb')

How many terms are in our vocabulary? Whenever interacting with the word vector dictionary, we use the `wv` attribute:

In [7]:
len(model.wv)

14732

Let's take a peek at the word vectors our model has learned. We can take a look at the individual words using the `index_to_key` attribute, and the word vectors themselves can be accessed with the `vectors` attribute:

In [8]:
model.wv.index_to_key[0]

'like'

In [9]:
model.wv.vectors[0]

array([-0.08018274,  0.00491095,  0.08784632,  0.01433069, -0.09794639,
       -0.09224576,  0.00622013,  0.24962428,  0.06723873, -0.05270278,
       -0.03648977, -0.06783219, -0.08407432, -0.0634554 ,  0.0175021 ,
       -0.11634977,  0.09794779,  0.20852275, -0.00848902, -0.22265333,
        0.08231014, -0.06419335,  0.09797097,  0.0162484 ,  0.14323813,
        0.0611587 ,  0.01721993, -0.03901093, -0.04347777,  0.00503724,
       -0.03510949, -0.12510447,  0.20113452, -0.00120747, -0.01262403,
       -0.20425951,  0.2812798 , -0.07498191, -0.07580847, -0.06849664,
       -0.17727983,  0.00944687, -0.09995916, -0.17946193,  0.08719696,
        0.11549717, -0.12986168,  0.01182884, -0.08035369,  0.00328454,
       -0.06672981, -0.07913686, -0.01926371, -0.02746153,  0.05087628,
        0.1255642 ,  0.15487921, -0.22199556, -0.06654175,  0.20550871,
        0.03788844, -0.14240259,  0.12153263, -0.04456891,  0.08859289,
        0.09279501,  0.03287575,  0.15026289,  0.02606961, -0.09

Looking at it won't make a whole lot of sense to us! It's just a bunch of numbers. However, we can do semantic operations on these vectors, such as getting related terms.

<a id='sim'></a>

# Word Similarity

With the information in our word embeddings model, we can try to find similarities between words that interest us (i.e. words that have a similar vector). Let's create a function that retrieves related terms to some input. We're going to use the `most_similar()` function in `gensim` as part of this helper function.

In [10]:
def get_most_similar_terms(model, token, topn=20):
    """Look up the top N most similar terms to the token."""
    for word, similarity in model.wv.most_similar(positive=[token], topn=topn):
        print(f"{word}: {round(similarity, 3)}")

In [11]:
get_most_similar_terms(model, 'asshole')

veer: 0.625
asshole-: 0.623
nahyou’re: 0.618
redflag: 0.618
resounding: 0.605
assholeishness: 0.605
esh.your: 0.605
esh.he: 0.602
esh.i: 0.597
dickwad: 0.596
ytayou’re: 0.594
dickbag: 0.593
this:>i: 0.591
esh.she: 0.588
fuckwad: 0.587
arsehole: 0.585
nutjob: 0.585
yta.you're: 0.584
opportunist: 0.584
infact: 0.584


Here are some other terms. What else interests you?

In [12]:
get_most_similar_terms(model, 'empathy')

compassion: 0.697
rationality: 0.573
lacking: 0.564
modicum: 0.559
lack_empathy: 0.551
humility: 0.547
patience: 0.545
self_awareness: 0.537
devoid: 0.529
empathic: 0.525
empathetic: 0.523
empathize: 0.523
coping_skill: 0.522
self_reflection: 0.508
empathise: 0.505
tact: 0.505
blindsided: 0.503
inadequacy: 0.502
cold_hearted: 0.502
inferiority: 0.498


In [13]:
get_most_similar_terms(model, 'relationship')

amicably: 0.619
polyamory: 0.599
exclusivity: 0.592
transparency: 0.592
polyamorous: 0.589
fidelity: 0.588
irrevocably: 0.585
relationships: 0.583
marriage: 0.582
trusting: 0.581
coparent: 0.574
infatuate: 0.574
sexually_compatible: 0.572
ltr: 0.566
reestablish: 0.565
friendship: 0.564
mono: 0.563
openness: 0.556
monogamy: 0.553
overthink: 0.548


In [14]:
get_most_similar_terms(model, 'power')

differential: 0.445
power_imbalance: 0.42
wield: 0.417
intimidation: 0.407
manipulate: 0.402
dominance: 0.386
ideological: 0.384
institutional: 0.383
tripping: 0.374
dynamic: 0.372
perpetual: 0.372
deity: 0.371
authority: 0.37
domineering: 0.369
tactic: 0.369
exert: 0.368
chivalry: 0.367
leverage: 0.366
narcs: 0.365
seniority: 0.362


In [15]:
get_most_similar_terms(model, 'man')

woman: 0.643
men: 0.594
ogle: 0.547
subservient: 0.538
cisgender: 0.526
harassed: 0.524
het: 0.521
butch: 0.52
femme: 0.518
androgynous: 0.518
cucke: 0.517
shouldn't: 0.515
middle_aged: 0.513
misandry: 0.513
transman: 0.511
macho: 0.511
dude: 0.511
hes: 0.508
monolith: 0.508
abomination: 0.507


In [16]:
get_most_similar_terms(model, 'woman')

man: 0.643
men: 0.591
women: 0.573
androgynous: 0.573
transwoman: 0.569
ogle: 0.563
cisgender: 0.555
womanhood: 0.554
transman: 0.552
monolith: 0.539
ntabee: 0.539
fetishization: 0.533
butch: 0.53
het: 0.527
brunette: 0.527
femme: 0.526
mans: 0.522
misandry: 0.522
petite: 0.52
middle_aged: 0.52


# Visualizing High Dimensional Spaces with $t$-SNE

The word embeddings we created are what's called a **high-dimensional representation** of the text. That is, we take a word in the corpus, and represent it using, in this case, 300 numbers. We can plot 3 numbers at a time - that's in 3 dimensions - but there's no way for humans to visualize something in a 300-dimensional space. 

So, **dimensionality reduction** is a big part of machine learning. How can we take vectors that are 300-dimensional, and visualize them in 2-dimensions, while keeping the structure between vectors the same? How can we reduce the dimensionality?

One of the most popular methods for dimensionality reduction is called $t$-SNE ($t$-Distributed Stochastic Neighbor Embedding). If you want to read more about this algorithm, [here](https://lvdmaaten.github.io/tsne/) is a good starting point. 

Roughly, $t$-SNE tries to keep the relative distances between points as closely as possible in both high-dimensional and low-dimensional space. We can thus visualize our embeddings, which may reveal semantic and syntactic trends in the data.

One thing we can do is to look for relationships between particular word pairs. 

In [17]:
words = ['asshole', 'jerk', 'rude', 'inconsiderate',
         'angry', 'upset', 'sad', 'happy', 'frustrated',
         'friend', 'family', 'partner', 'coworker', 'neighbor',
         'apologize', 'forgive', 'ignore', 'confront', 'compromise',
         'right', 'wrong', 'justified', 'unjustified', 'fair', 'unfair',
         'honesty', 'loyalty', 'respect', 'kindness', 'empathy']

# Extract the word vectors
word_vectors = np.array([model.wv[word] for word in words])

In [18]:
# If you get an ImportError in the line tsne=TSNE(), you might need to install scikit-learn:
# %pip install -U scikit-learn 

In [19]:
# Reduce dimensionality using t-SNE
tsne = TSNE(n_components=2, random_state=2, perplexity=2)
reduced_vectors = tsne.fit_transform(word_vectors)

In [20]:
# Store the t-SNE vectors
words_df = pd.DataFrame(reduced_vectors,
                            index=pd.Index([word for word in words]),
                            columns=['x', 'y'])

We're going to visualize the 2-dimensional space using a package called `bokeh`. This package allows for some degree of interactivity: we can go over each point and dynamically get information about the word denoting that vector.

In [21]:
# You may need to install bokeh if the following cell does not run
# %pip install bokeh

In [24]:
import bokeh
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, LabelSet

output_notebook()
bokeh.io.output_notebook()

Loading BokehJS ...

Loading BokehJS ...

In [40]:
# Add our DataFrame as a ColumnDataSource for Bokeh
plot_data = ColumnDataSource(words_df)

# Create the plot and configure the title, dimensions, and tools
tsne_plot = figure(title='t-SNE Word Embeddings')

# Add a hover tool to display words on roll-over
tsne_plot.add_tools(HoverTool(tooltips='@index'))

# Draw the words as circles on the plot
tsne_plot.circle('x', 'y',
                 source=plot_data,
                 color='blue',
                 size=10,
                 hover_line_color='black')

# Add labels to the points
labels = LabelSet(x='x', y='y', text='index', level='glyph',
                  x_offset=5, y_offset=5, source=plot_data,
                  render_mode='canvas')
tsne_plot.add_layout(labels)

# Engage!
show(tsne_plot)

Now let's use $t$-SNE to take **all** the word vectors.

In [41]:
tsne = TSNE(init='pca', learning_rate='auto')
X_tsne = tsne.fit_transform(model.wv.vectors)

We have our low dimensional representation. Now, let's store the 2 dimensions in a dataframe, with the word as the index:

In [42]:
# Store the t-SNE vectors
tsne_df = pd.DataFrame(X_tsne,
                       index=pd.Index(model.wv.index_to_key),
                       columns=['x', 'y'])

In [43]:
# Create some filepaths to save our model
tsne_path = '../../data/tsne_model'
tsne_df_path = '../../data/tsne_df.pkl'

In [44]:
# Save to disk
with open(tsne_path, 'wb') as f:
    pickle.dump(X_tsne, f)

tsne_df.to_pickle(tsne_df_path)

Here's a convenient code block to load this data, to start from this point:

In [45]:
with open(tsne_path, 'rb') as f:
    X_tsne = pickle.load(f)
    
tsne_df = pd.read_pickle(tsne_df_path)

In [46]:
# Add our DataFrame as a ColumnDataSource for Bokeh
plot_data = ColumnDataSource(tsne_df)

# Create the plot and configure the title, dimensions, and tools
tsne_plot = figure(title='t-SNE Word Embeddings')

# Add a hover tool to display words on roll-over
tsne_plot.add_tools(HoverTool(tooltips='@index') )

# Draw the words as circles on the plot
tsne_plot.circle('x', 'y',
                 source=plot_data,
                 color='blue',
                 line_alpha=0.2,
                 fill_alpha=0.1,
                 size=10,
                 hover_line_color='black')

# Engage!
show(tsne_plot)

<a id='bias'></a>

# Discovering Language Bias With Word Embeddings

The vector representations of word Embeddings can be used for all kinds of tasks. One such task is discovering **language bias**: language that is normatively oriented towards certain people, ideas, or things. 

In order to do this, we first need to postulate **concepts** that we suspect a bias towards. For example, people often associate words with masculinity or femininity (think of "fireman" vs "nurse"). This reflects both a normative construction of a gender binary, as well as stereotypes *within* that binary. We can attempt to quantify this directly using notions of distance with word embeddings.

We first define **target concepts** for the terms we aim to identify biases towards (e.g., "male" and "female"). Then, we can then compute relative similarities of other word vectors – particularly, words that act as evaluative attributes such as "strong" and "sensitive". These words can be categorised through clustering algorithms and labeled through a semantic analysis system into more general (conceptual) biases, yielding a broad picture of the biases present in a discourse community.

At the bottom of this notebook you'll find some [existing target sets](#target) that have been used in the literature on language biases.

## Obtaining Biases Towards Target Concepts

We will run our method of finding biased words towards our target sets. We will be using functions in an external file - `utils.py` - which you are free to look through if you're interested. Otherwise, feel free to use the functions as desired.

Given a vocabulary and two sets of target words (such as, in this case, those for *women* and *men*), we rank the words from least to most biased. As such, we obtain two ordered lists of the most biased words towards each target set, obtaining an overall view of the bias distribution in that particular community with respect to those two target sets. 

The approach is as follows:
- We calculate the centroid of a target set by averaging the embedding vectors in our target set (e.g. the vectors for `he, son, his, him, father, male` for our target concept `male`);
- We calculate the cosine similarity between the vectors for all words in our vocabulary as compared to our two centroids (we also apply POS-filtering to only work with parts of speech we expect to be relevant);
- We use a threshold based on standard deviation to determine how severe a bias needs to be before we include it;
- We rank the words in the vocabulary of our word embeddings model based on their bias towards either target concept.

The implicit hypothesis here is that word vectors closer to the centroid of a target concept are used more similarly with respect that concept. This, in effect, is an effort to quantify the bias of words in the discourse of a community.

In [29]:
# Import function to calculate biased words
from utils import calculate_biased_words

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/tomvannuenen/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/tomvannuenen/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Here are our target concept vectors. This is a critical choice: the centroid of these represents the concept at hand. What happens if you change the words included?

In [30]:
target1 = ['sister' , 'female' , 'woman' , 'girl' , 'daughter' , 'she' , 'hers' , 'her']
target2 = ['brother' , 'male' , 'man' , 'boy' , 'son' , 'he' , 'his' , 'him'] 

In [31]:
model = Word2Vec.load('../../data/aita_comments.emb')

In [32]:
[b1, b2] = calculate_biased_words(model, target1, target2, 4)

Let's print some biases. Here you see the most-biased words towards our target concepts (1 being *women*, 2 being *men*).

In [33]:
print('Biased words towards target set 1')
print([word for word in b1.keys()])

Biased words towards target set 1
['people', 'body', 'personal', 'involve', 'concern', 'shame', 'creepy', 'valid', 'form', 'detail', 'threaten', 'coworker', 'zero', 'begin', 'birth_control', 'privacy', 'trauma', 'gain', 'painful', 'hurtful', 'social_medium', 'patient', 'intimate', 'intent', 'vulnerable', 'skinny', 'ntayou', 'controlling', 'self_esteem', 'useful', 'cross_line', 'dodge_bullet', 'unsafe', 'wig', 'self_conscious', 'unattractive', 'damaging', 'pushy', 'iud', 'risky', 'downright', 'workout', 'shaming', 'straighten']


In [34]:
print('Biased words towards target set 2')
print([word for word in b2.keys()] )

Biased words towards target set 2
['leave', 'deal', 'spend', 'responsibility', 'pass', 'god', 'argue', 'plenty', 'split', 'separate', 'understandable', 'contribute', 'earn', 'punishment', 'bro', 'vote', 'buddy', 'btw', 'mate', 'duty', 'vs', 'got', 'pride', 'grandfather', 'daddy', '😂', 'lord', 'grandpa', 'tonight', 'dear', 'sin', 'golden', 'lame', 'halloween', 'heaven']


## Visualizing Biases using $t$-SNE

We now return to our dimensionality reduction technique, $t$-SNE, to try and visualize these biased words in a 2D space using Bokeh.

In [35]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.manifold import TSNE
%matplotlib inline

In [36]:
with open(tsne_path, 'rb') as f:
    X_tsne = pickle.load(f)
    
tsne_df = pd.read_pickle(tsne_df_path)

In [37]:
# Convert biased term keys to arrays
target1_idx = np.array([model.wv.key_to_index[key] for key in b1.keys()])
target2_idx = np.array([model.wv.key_to_index[key] for key in b2.keys()])

In [38]:
# Find t-sne values for the biased sets
X_target1 = X_tsne[target1_idx]
X_target2 = X_tsne[target2_idx]

In [39]:
from bokeh.io import show, output_notebook, output_file
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LabelSet

# Set up the Bokeh plot
output_notebook()

p = figure()

# Create ColumnDataSource for X_target1 (blue)
source1 = ColumnDataSource(data=dict(x=X_target1[:, 0], y=X_target1[:, 1], label=[model.wv.index_to_key[idx] for idx in target1_idx]))

# Create ColumnDataSource for X_target2 (red)
source2 = ColumnDataSource(data=dict(x=X_target2[:, 0], y=X_target2[:, 1], label=[model.wv.index_to_key[idx] for idx in target2_idx]))

# Add scatter plot for X_target1 (blue)
p.scatter(x='x', y='y', color='blue', size=8, source=source1)

# Add scatter plot for X_target2 (red)
p.scatter(x='x', y='y', color='red', size=8, source=source2)

# Add labels for X_target1
labels1 = LabelSet(x='x', y='y', text='label', x_offset=6, y_offset=3, source=source1, render_mode='canvas')
p.add_layout(labels1)

# Add labels for X_target2
labels2 = LabelSet(x='x', y='y', text='label', x_offset=6, y_offset=3, source=source2, render_mode='canvas')
p.add_layout(labels2)

# Show the plot
show(p)

Loading BokehJS ...

## 💭 Reflection

Note that these binary target concepts are often a product of ideology and normativity in society: the gender binary is a good example. When checking for biases towards certain concepts, make sure you consider the fact that you are the one creating / reproducing these concepts, and that you may be reinforcing a constructed binary!

Also note that determining your own target concepts and biases is a **iterative** process. Try changing some of the words in the target concepts to see how the biased words and plot change, and discuss with your group what you think makes for a coherent and robust target set.

<a id='target'></a>
# Existing Target Sets

Here are some other target sets that have been previously used in the literature:

* *Gender target sets taken from Nosek, Banaji, and Greenwald 2002.*
    - Female: `sister, female, woman, girl, daughter, she, hers, her`.
    - Male: `brother, male, man, boy, son, he, his, him`.
* *Religion target sets taken from Garg et al. 2018.*
    - Islam: `allah, ramadan, turban, emir, salaam, sunni, koran, imam, sultan, prophet, veil, ayatollah, shiite, mosque, islam, sheik, muslim, muhammad`.
    - Christianity: `baptism, messiah, catholicism, resurrection, christianity, salva-tion, protestant, gospel, trinity, jesus, christ, christian, cross,catholic, church`.
* *Racial target sets taken from Garg et al. 2017*
    - White last names: `harris, nelson, robinson, thompson, moore, wright, anderson, clark, jackson, taylor, scott, davis, allen, adams, lewis, williams, jones, wilson, martin, johnson`.
    - Hispanic last names: `ruiz, alvarez, vargas, castillo, gomez, soto,gonzalez, sanchez, rivera, mendoza, martinez, torres, ro-driguez, perez, lopez, medina, diaz, garcia, castro, cruz`.
    - Asian last names: `cho, wong, tang, huang, chu, chung, ng,wu, liu, chen, lin, yang, kim, chang, shah, wang, li, khan,singh, hong`.
    - Russian last names: `gurin, minsky, sokolov, markov, maslow, novikoff, mishkin, smirnov, orloff, ivanov, sokoloff, davidoff, savin, romanoff, babinski, sorokin, levin, pavlov, rodin, agin`.
* *Career/family target sets taken from Garg et al. 2018.*
    - Career: `executive, management, professional, corporation, salary, office, business, career`.
    - Family: `home, parents, children, family, cousins, marriage, wedding, relatives.Math: math, algebra, geometry, calculus, equations, computation, numbers, addition`.
* *Arts/Science target sets taken from Garg et al. 2018.*
    - Arts: `poetry, art, sculpture, dance, literature, novel, symphony, drama`.
    - Science: `science, technology, physics, chemistry, Einstein, NASA, experiment, astronomy`.

### Sources

Nosek, B. A., Banaji, M. R., & Greenwald, A. G. (2002). Harvesting implicit group attitudes and beliefs from a demonstration web site. Group Dynamics, 6(1), 101–115. https://doi.org/10.1037/1089-2699.6.1.101

Garg, N., Schiebinger, L., Jurafsky, D., & Zou, J. (2017). Word Embeddings Quantify 100 Years of Gender and Ethnic Stereotypes, 1–33.